### Aggregate the final glaciers that have been chosen for our study.

Prior to this:
1. we automated the first-pass of by selecting glaciers according to debris-covered area in herreid et al 2020
2. we manually inspected those glaciers in QGIS, with multiple imagery basemaps, to select glaciers that we thought had significant debris cover
3. we then created a sentienl-2 cloud-free snow-free mosaic in GEE, which we used to further sub-select glaciers that had a significant portion of debris cover. this generally meant they had at least a 500 m stretch of continuous debris cover with no exposed glacier ice within this section.

Now we have a csv that holds those rgi IDs, so we are going to create a final RGI subset with only those outlines (and also subset the herreid datasets as well)

In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import pandas as pd
from datetime import timedelta
from datetime import datetime

In [2]:
# define folder and file paths
folder_HMA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"HMA2")
folder_debris = os.path.join(folder_HMA, 'Datasets', 'debris cover')
folder_rgi = os.path.join(folder_HMA, 'Datasets', 'RGI')
folder_lakes = os.path.join(folder_HMA, 'Datasets', 'lake inventories')

In [3]:
# open csvs that allow us to link rgiids between v6 and v7
link_13 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-13_central_asia-rgi6_links.csv'))
link_14 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-14_south_asia_west-rgi6_links.csv'))
link_15 = pd.read_csv(os.path.join(folder_rgi, 'links', 'RGI2000-v7.0-G-15_south_asia_east-rgi6_links.csv'))
# link_15.head()

In [4]:
##### open RGI v7 and herreid2020 data

# herreid debris cover, herreid glacier extents, rgi v7, and then fix 3d geometry
h_dc_15 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '15SouthAsiaEast', '15SouthAsiaEast_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_15 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '15SouthAsiaEast', '15SouthAsiaEast_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_15 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-15_south_asia_east', 'RGI2000-v7.0-G-15_south_asia_east.shp')).to_crs("ESRI:102025")
rgi_15['geometry'] = gpd.GeoSeries.from_wkb(rgi_15.to_wkb(output_dimension=2).geometry)

h_dc_13 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '13CentralAsia', '13CentralAsia_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_13 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '13CentralAsia', '13CentralAsia_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_13 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-13_central_asia', 'RGI2000-v7.0-G-13_central_asia.shp')).to_crs("ESRI:102025")
rgi_13['geometry'] = gpd.GeoSeries.from_wkb(rgi_13.to_wkb(output_dimension=2).geometry)

h_dc_14 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '14SouthAsiaWest', '14SouthAsiaWest_minGl1km2_debrisCover.shp')).to_crs("ESRI:102025")
h_g_14 = gpd.read_file(os.path.join(folder_debris, 'Herreid2020', '14SouthAsiaWest', '14SouthAsiaWest_minGl1km2.shp')).to_crs("ESRI:102025")
rgi_14 = gpd.read_file(os.path.join(folder_rgi, 'v7', 'G', 'RGI2000-v7.0-G-14_south_asia_west', 'RGI2000-v7.0-G-14_south_asia_west.shp')).to_crs("ESRI:102025")
rgi_14['geometry'] = gpd.GeoSeries.from_wkb(rgi_14.to_wkb(output_dimension=2).geometry)

In [5]:
# concat everything into single dfs
h_dc_all = gpd.GeoDataFrame(pd.concat([h_dc_13, h_dc_14, h_dc_15]))
h_g_all = gpd.GeoDataFrame(pd.concat([h_g_13, h_g_14, h_g_15]))
rgi_all = gpd.GeoDataFrame(pd.concat([rgi_13, rgi_14, rgi_15]))
rgi_all.shape

(131762, 29)

In [15]:
# now open our list of glaciers that we've hand picked
path_open = os.path.join(folder_HMA, "Datasets", 'AOI definition', 'pass2_results.csv')
rgi_selected_ids = pd.read_csv(path_open)
rgi_selected_ids.shape

(831, 1)

In [16]:
# grab these from rgi gdf
rgi_selected = rgi_all[ rgi_all['rgi_id'].isin( rgi_selected_ids['rgi_id']) ]
rgi_selected.shape

(831, 29)

In [17]:
# buffer rgi by 5km
rgi_5km_buffer = gpd.GeoDataFrame(crs=rgi_selected.crs,
                       geometry=[rgi_selected.buffer(5000).unary_union])

In [18]:
# save these
out_path = os.path.join(folder_HMA, "Datasets", 'AOI definition', 'Final AOI', "AOI_rgi.shp")
rgi_selected.to_file(out_path)

out_path = os.path.join(folder_HMA, "Datasets", 'AOI definition', 'Final AOI', "AOI_5km.shp")
rgi_5km_buffer.to_file(out_path)

In [19]:
# clip herreid debris-cover to this 5km buffer and save
h_dc_5km = gpd.GeoDataFrame(crs=rgi_selected.crs,
                       geometry=[gpd.overlay(rgi_5km_buffer, h_dc_all, how='intersection').unary_union])

out_path = os.path.join(folder_HMA, "Datasets", 'AOI definition', 'Final AOI', "herreid_dc_5km.shp")
h_dc_5km.to_file(out_path)